In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import torch
from modelscope import AutoTokenizer, AutoModelForCausalLM
from PIL import Image
#ckpt_path = "Shanghai_AI_Laboratory/internlm-xcomposer2-7b-4bit"
## 模型下载

ckpt_path = "Shanghai_AI_Laboratory/internlm-xcomposer2-7b"
if not os.path.exists(ckpt_path):
    from modelscope import snapshot_download
    model_dir = snapshot_download(ckpt_path)
else:
    print('has model')
tokenizer = AutoTokenizer.from_pretrained(ckpt_path, trust_remote_code=True)
# `torch_dtype=torch.float16` 可以令模型以 float16 精度加载，否则 transformers 会将模型加载为 float32，导致显存不足
model = AutoModelForCausalLM.from_pretrained(ckpt_path, torch_dtype=torch.float16,
                                            offload_folder="offloadFolder",
                                             trust_remote_code=True, device_map='auto')



model = model.eval() 


Downloading: 100%|██████████| 15.6k/15.6k [00:00<00:00, 4.36MB/s]
Downloading: 100%|██████████| 7.18k/7.18k [00:00<00:00, 15.3MB/s]
Downloading: 100%|██████████| 0.99k/0.99k [00:00<00:00, 8.69MB/s]
Downloading: 100%|██████████| 48.0/48.0 [00:00<00:00, 430kB/s]
Downloading: 100%|██████████| 7.37k/7.37k [00:00<00:00, 7.68MB/s]
Downloading: 100%|██████████| 175/175 [00:00<00:00, 1.01MB/s]
Downloading: 100%|██████████| 101k/101k [00:00<00:00, 10.5MB/s]
Downloading: 100%|██████████| 45.1k/45.1k [00:00<00:00, 24.1MB/s]
Downloading: 100%|██████████| 38.5k/38.5k [00:00<00:00, 13.6MB/s]
Downloading: 100%|██████████| 24.4k/24.4k [00:00<00:00, 7.50MB/s]
Downloading: 100%|██████████| 239k/239k [00:00<00:00, 18.9MB/s]
Downloading: 100%|█████████▉| 16.1G/16.1G [00:52<00:00, 330MB/s]
Downloading: 100%|██████████| 8.03k/8.03k [00:00<00:00, 16.7MB/s]
Downloading: 100%|██████████| 95.0/95.0 [00:00<00:00, 683kB/s]
Downloading: 100%|██████████| 8.96k/8.96k [00:00<00:00, 24.3MB/s]
Downloading: 100%|███████

Set max length to 4096


Downloading: 100%|██████████| 4.65k/4.65k [00:00<00:00, 5.84MB/s]
Downloading: 100%|██████████| 49.0/49.0 [00:00<00:00, 377kB/s]
Downloading: 100%|██████████| 512k/512k [00:00<00:00, 31.7MB/s]
Downloading: 100%|██████████| 316/316 [00:00<00:00, 2.82MB/s]
Downloading: 100%|█████████▉| 1.59G/1.59G [00:04<00:00, 376MB/s]
Downloading: 100%|██████████| 1.02k/1.02k [00:00<00:00, 7.14MB/s]
Downloading: 100%|██████████| 389/389 [00:00<00:00, 2.03MB/s]
Downloading: 100%|█████████▉| 1.59G/1.59G [00:09<00:00, 189MB/s]
Downloading: 100%|██████████| 2.12M/2.12M [00:00<00:00, 66.0MB/s]
Downloading: 100%|██████████| 844/844 [00:00<00:00, 3.57MB/s]
Downloading: 100%|██████████| 842k/842k [00:00<00:00, 29.8MB/s]
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.class_embedding: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items 

Position interpolate from 24x24 to 16x16


In [7]:
import time
t1=time.time()
img_path_list = [
    'gushi2.jpg',
]
images = []
for img_path in img_path_list:
    image = Image.open(img_path).convert("RGB")
    image = model.vis_processor(image)
    images.append(image)
image = torch.stack(images)
query = '<ImageHere>请根据图片写一篇作文：我最喜欢的动物。要求：选准角度，确定立意，明确文体，自拟标题。'
query ='''<ImageHere>
# 角色
你是一个才华横溢的儿童故事创造者，利用照片中的故事画册中的主角和两个场景，再创作一个吸引小朋友的儿童故事。此次创作故事的主题为 "让宝宝自主刷牙"。请按下面步骤进行创作：

## 步骤

### 步骤1: 解析图片

- 识别图片中的故事画册的主角和情境，理解和识别他们的特点。
- 将这些特点和情境融入到故事创作中。
- 创作的故事的主角使用照片中的角色


### 步骤2: 创作故事

- 把 "让宝宝自主刷牙" 这个主题巧妙地编织入故事当中，让故事既有趣味性又有启发性。
- 图片中的主角充分融入到创作的故事中
- 使用生动的语言和引人入胜的情节，使故事更具吸引力。

### 步骤3: 故事呈现

- 用独特的方式讲述故事，让小朋友们喜欢听。
- 能够以适合小朋友理解的方式，讲述故事的精彩片段。

## 限制：
- 故事中不能含有任何不适宜儿童的内容。
- 故事需要紧扣主题，不能偏离 “让宝宝自主刷牙” 这个主题。
- 故事长度适中，不宜过长。
'''
with torch.cuda.amp.autocast():
    response, history = model.chat(tokenizer, query=query, image=image, history=[], do_sample=False)
print(response)
print(time.time()-t1)

标题是："小马和小松鼠的刷牙冒险"

故事开始于一个阳光明媚的早晨，在一片绿油油的草地上，一只名叫小白的小马正在悠闲地吃草。突然，他听到了一阵急促的呼救声，原来是他的好朋友小松鼠小红被困在了树洞里。

小白非常着急，他知道只有一种方法可以救出小红——那就是用一根绳子系住自己的尾巴，让小红抓住绳子爬出来。但是，小白从来没有试过这种方法，他有些害怕。

就在这时，小白的妈妈走了过来，她看到了小白和小红的情况，便对小白说：“小白，你记得我们每天都要刷牙吗？”小白点了点头。妈妈接着说：“那你知道为什么我们要刷牙吗？因为我们要保护我们的牙齿，不让它们生病。现在，你的朋友小红需要帮助，你可以像保护牙齿一样勇敢地去帮助他。”

听了妈妈的话，小白感到勇气倍增。他决定尝试用自己的尾巴来救出小红。小白把绳子系在自己的尾巴上，然后让小红抓住绳子爬了出来。他们两个都松了一口气，互相拥抱了一下。

从那天起，小白和小红成为了最好的朋友。他们一起玩耍、探险，还一起学习如何保护牙齿。每次刷牙的时候，小白都会想起那次勇敢的救援行动，他觉得保护牙齿就像保护自己最好的朋友一样重要。

这个故事告诉我们，保护牙齿就像是保护我们的朋友一样重要。我们应该每天刷牙，保持口腔卫生，这样才能拥有健康的牙齿和快乐的笑容。
24.974496364593506
